In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("parulpandey/palmer-archipelago-antarctica-penguin-data")

print("Path to dataset files:", path)

/home/ikx/Desktop/NSU_courses/NSU_Statistic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/ikx/.cache/kagglehub/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data/versions/1


In [2]:
import pandas as pd
from tabulate import tabulate
import seaborn as sns

In [3]:
df = pd.read_csv(path + '/penguins_size.csv')

In [ ]:
def dataframe_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    """
    Выполняет очистку датасета от пропусков и некорректных значений.
    """
    df = df.dropna()
    df = df[df['sex'] != '.']
    return df

def dataset_report(df: pd.DataFrame, top_n: int = None):
    """
    Генерация сводной таблицы по датафрейму:
    - для числовых признаков: min, max, mean
    - для категориальных признаков: распределение (все или top_n значений)
    
    :param df: DataFrame для анализа
    :param top_n: ограничение на количество выводимых значений (например, top_n=5)
    """
    info = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            info.append([
                col,
                str(df[col].dtype),
                df[col].isna().sum(),
                round(df[col].isna().mean()*100, 2),
                df[col].nunique(),
                df[col].min(),
                df[col].max(),
                round(df[col].mean(), 2) if df[col].notna().any() else "-",
                "-"
            ])
        else:
            # распределение категориального признака
            vc = df[col].value_counts(dropna=False)
            if top_n is not None and len(vc) > top_n:
                top = vc.head(top_n)
                other_sum = vc.iloc[top_n:].sum()
                dist = dict(top)
                dist["Other"] = other_sum
            else:
                dist = dict(vc)
            dist_str = ", ".join([f"{k}: {v}" for k, v in dist.items()])
            info.append([
                col,
                str(df[col].dtype),
                df[col].isna().sum(),
                round(df[col].isna().mean()*100, 2),
                df[col].nunique(),
                "-",
                "-",
                "-",
                dist_str
            ])
    
    headers = [
        "Колонка", "Тип", "Пропуски", "% пропусков",
        "Уникальные", "Мин", "Макс", "Среднее", "Распределение"
    ]
    
    print("Размер датасета:", df.shape)
    print(tabulate(info, headers=headers, tablefmt="grid"))



In [ ]:
"""
Статистика по исходному датасету
"""
dataset_report(df)

Размер датасета: (344, 7)
+-------------------+---------+------------+---------------+--------------+--------+--------+-----------+-----------------------------------------+
| Колонка           | Тип     |   Пропуски |   % пропусков |   Уникальные | Мин    | Макс   | Среднее   | Распределение                           |
+===================+=========+============+===============+==============+========+========+===========+=========================================+
| species           | object  |          0 |          0    |            3 | -      | -      | -         | Adelie: 152, Gentoo: 124, Chinstrap: 68 |
+-------------------+---------+------------+---------------+--------------+--------+--------+-----------+-----------------------------------------+
| island            | object  |          0 |          0    |            3 | -      | -      | -         | Biscoe: 168, Dream: 124, Torgersen: 52  |
+-------------------+---------+------------+---------------+--------------+--------+--

In [ ]:
"""
Статистика по чистому датасету
"""
df = dataframe_cleaning(df)
dataset_report(df)

Размер датасета: (333, 7)
+-------------------+---------+------------+---------------+--------------+--------+--------+-----------+-----------------------------------------+
| Колонка           | Тип     |   Пропуски |   % пропусков |   Уникальные | Мин    | Макс   | Среднее   | Распределение                           |
+===================+=========+============+===============+==============+========+========+===========+=========================================+
| species           | object  |          0 |             0 |            3 | -      | -      | -         | Adelie: 146, Gentoo: 119, Chinstrap: 68 |
+-------------------+---------+------------+---------------+--------------+--------+--------+-----------+-----------------------------------------+
| island            | object  |          0 |             0 |            3 | -      | -      | -         | Biscoe: 163, Dream: 123, Torgersen: 47  |
+-------------------+---------+------------+---------------+--------------+--------+--